In [ ]:
!pip3 install thirdai --upgrade
!pip3 install datasets==2.6.2
!pip3 install pydantic==1.10.10
!pip3 install gradio



In [6]:
import thirdai
thirdai.licensing.deactivate()

In [7]:
thirdai.licensing.activate("1FB7DD-CAC3EC-832A67-84208D-C4E39E-V3")

In [8]:
import pandas as pd
from thirdai.demos import to_udt_input_batch
import json

def load_data(output_filename, split=None, return_inference_batch=False):
    if split is None:
        # Load the CSV file into a DataFrame
        df = pd.read_csv(output_filename, sep='\t')
    else:
        # Process and save the data
        with open('main.json', encoding='utf-8') as f:
          data=json.load(f)
        df = pd.DataFrame(data)
        print(df)

        # Save the data to a CSV file in a tab-separated format
        df.to_csv(output_filename, index=False, sep='\t')

    if return_inference_batch:
        inference_batch = to_udt_input_batch(df[["headline"]].sample(frac=1).iloc[:5])
        return inference_batch


In [9]:
train_filename = "stock_train.csv"
test_filename = "stock_test.csv"
load_data(train_filename, split='train')
inference_batch = load_data(test_filename, split='test', return_inference_batch=True)

                                              headline  rating
0         Tesla Shares Sink as I.P.O. Excitement Wanes       0
1    Tesla Motors Shares Drop Below Initial Sale Price       0
2    Tesla May Get Some Company: The Supercar Goes ...       1
3    Tesla Says It Is Testing an Electric Car Proto...       1
4               Toyota and Tesla Plan an Electric RAV4       1
..                                                 ...     ...
462  China’s Bullying Is Becoming a Danger to the W...       0
463  F.T.C. Sues to Stop Blockbuster Chip Deal Betw...       1
464                              A Tale of Redemptions       0
465  Is it time for big in-person events again? Org...       0
466                  Was the Market Sell-Off Overdone?       1

[467 rows x 2 columns]
                                              headline  rating
0         Tesla Shares Sink as I.P.O. Excitement Wanes       0
1    Tesla Motors Shares Drop Below Initial Sale Price       0
2    Tesla May Get Some Company

In [10]:
from thirdai import bolt

model = bolt.UniversalDeepTransformer(
    data_types={
        "headline": bolt.types.text(),
        "rating": bolt.types.categorical()
    },
    target="rating",
    n_target_classes=3,
    delimiter='\t'

)

In [11]:
model.train(train_filename, epochs=5, learning_rate=0.01, metrics=["categorical_accuracy"])

{'train_categorical_accuracy': [0.3790149986743927,
  0.6102783679962158,
  0.7087794542312622,
  0.8436830639839172,
  0.9443255066871643],
 'epoch_times': [0.0, 0.0, 0.0, 0.0, 0.0]}

In [12]:
model.evaluate(test_filename, metrics=["categorical_accuracy"]);

In [13]:
save_location = "sentiment_analysis.model"

# Saving
model.save(save_location)

# Loading
model = bolt.UniversalDeepTransformer.load(save_location)

In [14]:
model = bolt.UniversalDeepTransformer.load("sentiment_analysis.model")

In [15]:
import gradio as gr


# def predict(text):
#     dict = {}
#     act = model.predict({"headline": text})
#     class_id = np.argmax(act, axis=0)
#     act = act.tolist()
#     if model.class_name(class_id) == "1":
#         dict["positive"] = act[class_id]
#         dict["negative"] = act[1 - class_id]
#     else:
#         dict["positive"] = act[1 - class_id]
#         dict["negative"] = act[class_id]
#     print(dict)
#     return dict

def predict(text):
    result_dict = {}
    class_labels = {
        'noChange': 0,
        'priceUp': 1,
        'priceDown': 2,
    }

    act = model.predict({"headline": text})
    act = act.squeeze()  # Remove any singleton dimensions
    act = act.tolist()   # Convert the ndarray to a list
    
    for label, index in class_labels.items():
        result_dict[label] = act[index]

    return result_dict




examples = [
     ["Enterprise Performance Monitoring Market to Show Incredible Revenue Growth by 2030"],
    ["IBM joins AI race with Watsonx, all-in-one tool for businesses"],
    ["IBM stays quiet on dismissals, even managers don't know quantum of job cuts"]
]

demo = gr.Interface(
    fn=predict,
    inputs=[gr.inputs.Textbox(lines=1, label="Input text")],
    outputs=gr.Label(),
    examples=examples,
)

demo.launch()

d:\Python\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\admin\AppData\Local\Temp\ipykernel_17888\1818920861.py:46: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=[gr.inputs.Textbox(lines=1, label="Input text")],
C:\Users\admin\AppData\Local\Temp\ipykernel_17888\1818920861.py:46: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=[gr.inputs.Textbox(lines=1, label="Input text")],
C:\Users\admin\AppData\Local\Temp\ipykernel_17888\1818920861.py:46: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  inputs=[gr.inputs.Textbox(lines=1, label="Input text")],


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [17]:
from flask import Flask, request, jsonify,render_template
from flask_cors import CORS
app=Flask(__name__)
CORS(app)

@app.route("/", methods=["GET", "POST"])
def chatbot_interface():
    newsDescription= str(request.args.get('newsDescription'))
    try:

        
        bot_message = predict(newsDescription)
        return jsonify(bot_message)
    except:
        return jsonify("{'priceUp': 0.21144632995128632,'priceDown': 0.5135731428861618,'noChange': 0.2135731428861618}")
    

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with stat


SystemExit: 1

C:\Users\admin\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3386: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
